In [1]:
import pandas as pd
import numpy as np
import pyarrow

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Q1

In [43]:
df = pd.read_parquet('./yellow_tripdata_2022-01.parquet')

In [3]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
df.shape

(2463931, 19)

In [5]:
len(list(df.columns))

19

# Q2

In [44]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration_min'] = df.duration.dt.total_seconds() / 60

In [7]:
df[['duration', 'duration_min']].head()

,duration,duration_min
0,0 days 00:17:49,17.816667
1,0 days 00:08:24,8.400000
2,0 days 00:08:58,8.966667
3,0 days 00:10:02,10.033333
4,0 days 00:37:32,37.533333


In [8]:
df.duration_min.std()

46.44530513776802

# Q3

In [45]:
df_outlies = df[(df.duration_min >= 1) & (df.duration_min <= 60)]

In [10]:
len(df_outlies) / len(df)

0.9827547930522406

# Q4

In [46]:
# df_outlies[['PULocationID', 'DOLocationID']].isnull().sum()
df_ohe = df_outlies[['PULocationID', 'DOLocationID']]

In [47]:
df_ohe = df_ohe.astype('str')
df_ohe.dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [48]:
df_ohe = df_ohe.to_dict(orient='records')

In [49]:
df_ohe[:5]

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'},
 {'PULocationID': '166', 'DOLocationID': '166'},
 {'PULocationID': '114', 'DOLocationID': '68'},
 {'PULocationID': '68', 'DOLocationID': '163'}]

In [55]:
dictV = DictVectorizer()
x_train = dictV.fit_transform(df_ohe)

In [51]:
x_train.shape

(2421440, 515)

In [52]:
len(dictV.feature_names_)

515

# Q5

In [18]:
y_train = df_outlies.duration_min.values

In [19]:
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

LinearRegression()

In [20]:
y_pred = lin_reg.predict(x_train)

In [21]:
mean_squared_error(y_train, y_pred, squared=False)

6.986190135979941

# Q6

In [59]:
df_feb = pd.read_parquet('./yellow_tripdata_2022-02.parquet')
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb['duration_min'] = df_feb.duration.dt.total_seconds() / 60
df_feb = df_feb[(df_feb.duration_min >= 1) & (df_feb.duration_min <= 60)]

df_ohe_test = df_feb[['PULocationID', 'DOLocationID']]
df_ohe_test = df_ohe_test.astype('str')
df_ohe_test = df_ohe_test.to_dict(orient='records')
x_test = dictV.transform(df_ohe_test)

y_test = df_feb.duration_min.values

In [60]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2421440, 515), (2918187, 515), (2421440,), (2918187,))

In [61]:
y_pred_test = lin_reg.predict(x_test)

In [62]:
mean_squared_error(y_test, y_pred_test, squared=False)

7.786389393934764